# DeepLog Model with Captum

In [16]:
# Using DeepAid for model interpretability 
#!pip install gdown
#!gdown 1mhraNt2Z8X6S6dFjjRN_hK3ph3sXPmjR
#!pip install prettytable

In [3]:
import numpy as np
from sklearn.metrics import confusion_matrix, roc_curve
import torch
import pandas as pd
def validate_by_rmse(rmse_vec,thres,label):
    pred = np.asarray([0] * len(rmse_vec))
    idx = np.where(rmse_vec>thres)
    pred[idx] = 1
    cnf_matrix = confusion_matrix(label, pred)
    FP = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix)  
    FN = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)
    TP = np.diag(cnf_matrix)
    TN = cnf_matrix.sum() - (FP + FN + TP)

    FP = FP.astype(float)
    FN = FN.astype(float)
    TP = TP.astype(float)
    TN = TN.astype(float)

    TPR = (TP/(TP+FN))[1]
    FPR = (FP/(FP+TN))[1]
    print("TPR:",TPR,"|FPR:",FPR)

    return pred

class Normalizer:
    def __init__(self, 
            dim, 
            normer="minmax",
            online_minmax=False): # whether fit_transform online (see Kitsune), *available only for normer="minmax"

        self.dim = dim # feature dimensionality
        self.normer = normer
        if self.normer == 'minmax':
            self.online_minmax = online_minmax
            self.norm_max = [-np.Inf] * self.dim
            self.norm_min = [np.Inf] * self.dim
        else:
            raise NotImplementedError # Implement other Normalizer here
        
    def fit_transform(self,train_feat):
        if self.normer == 'minmax':
            return self._minmax_fit_transform(train_feat)
        else:
            raise NotImplementedError # Implement other Normalizer here

    def transform(self,feat):
        if self.normer == 'minmax':
            return self._minmax_transform(feat)
        else:
            raise NotImplementedError # Implement other Normalizer here

    def restore(self,feat):
        if self.normer == 'minmax':
            return self._minmax_restore(feat)
        else:
            raise NotImplementedError # Implement other Normalizer here
        
    def _minmax_fit_transform(self,train_feat):
        if not self.online_minmax:
            self.norm_min = np.min(train_feat,axis=0)
            self.norm_max = np.max(train_feat,axis=0)
            norm_feat = (train_feat - self.norm_min) / (self.norm_max-self.norm_min+1e-10)
            return norm_feat
        else:
            norm_feat = []
            self.norm_max, self.norm_min = np.asarray(self.norm_max), np.asarray(self.norm_min)
            for i in range(len(train_feat)):
                x = train_feat[i]
                self.norm_max[x>self.norm_max] = x[x>self.norm_max]
                self.norm_min[x<self.norm_min] = x[x<self.norm_min]
                norm_feat.append((x - self.norm_min) / (self.norm_max-self.norm_min+1e-10))
            return np.asarray(norm_feat)

    def _minmax_transform(self, feat):
        norm_feat = (feat - self.norm_min) / (self.norm_max-self.norm_min+1e-10)
        return norm_feat

    def _minmax_restore(self, feat):
        denorm_feat = feat * (self.norm_max-self.norm_min+1e-10) + self.norm_min
        return denorm_feat
    


""" Deeplog tools """
def deeplogtools_seqformat(model, abnormal_data, num_candidates, index=0):
    import keras.utils.np_utils as np_utils
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    X = abnormal_data.copy()
    y, X = X[:,-1], np_utils.to_categorical(X[:,:-1])
    Output = model(torch.from_numpy(X).type(torch.float).to(device))
    TP_idx = []
    for i in range(len(Output)):
        output = Output[i]
        label = y[i]
        predicted = torch.argsort(output)[-num_candidates:]
        if label not in predicted:
            TP_idx.append(i)
    seq_feat = np_utils.to_categorical(abnormal_data[TP_idx])
    feat = seq_feat[index]
    seq = torch.from_numpy(feat[:-1,:]).to(device)
    label = torch.tensor(np.argmax(feat[-1])).unsqueeze(0).to(device)
    return seq,label, abnormal_data[TP_idx][index]

""" Multi LSTM tools """
def multiLSTM_seqformat(test_feat, seq_len = 5, index=0):
    import more_itertools

    X_test = more_itertools.windowed(test_feat[:,:],n=seq_len,step=1)
    X_test = np.asarray(list(X_test))
    y_test = np.asarray(test_feat[seq_len-1:])

    # print("X_test:",X_test.shape,"y_test:",y_test.shape)
    i = index
    interp_feat = y_test[i]
    seq_feat = np.asarray([X_test[i]]) 
    # print("seq_feat:",seq_feat.shape,"interp_feat:",interp_feat.shape)

    return seq_feat, interp_feat

In [4]:
import sys
# sys.path.insert(1,'/deepaid')

#sys.path.append('deepaid.deeplog')

sys.path.insert(1,'deepaid')

from deepaid.deeplog import *

from deepaid.deeplog import LSTM_onehot
import torch

In [5]:
# Import the model
model = torch.load("LSTM_onehot.pth.tar", map_location=torch.device('cpu'))
model.eval()

LSTM_onehot(
  (lstm): LSTM(28, 64, num_layers=2, batch_first=True)
  (fc): Linear(in_features=64, out_features=28, bias=True)
)

In [6]:
"""Step 2: Find an anomaly you are interested in"""
abnormal_data = np.load('deepaid/abnormal_data.npy')
idx = 100
seq, label, anomaly_timeseries = deeplogtools_seqformat(model, abnormal_data, num_candidates=9, index=idx)

In [7]:
"""Step 3: Create a DeepAID Interpreter"""

from timeseries_onehot import UniTimeseriesAID
feature_desc = np.load('deepaid/log_key_meanning.npy') # feature_description
my_interpreter = UniTimeseriesAID(model, feature_desc=feature_desc, class_num=28)

"""Step 4: Interpret your anomaly and show the result"""
interpretation = my_interpreter(seq, label)
my_interpreter.show_table(anomaly_timeseries, interpretation)

Successfully Initialize <Univariate Timeseries Interptreter> for Model <LSTM_onehot>

Visualize Interpretation (Table View)
+------+-------------------------------------+-------+------+-------------------------------------+
| Ano. |               Meaning               | Diff. | Ref. |               Meaning*              |
+------+-------------------------------------+-------+------+-------------------------------------+
|  4   |      Receiving blk* src&dest:*      |       |  4   |      Receiving blk* src&dest:*      |
|  10  |  PktResponder* for blk* terminating |       |  10  |  PktResponder* for blk* terminating |
|  9   |       PktResponder* Exception       |       |  9   |       PktResponder* Exception       |
|  13  |  Exception in receiveBlock for blk* |       |  13  |  Exception in receiveBlock for blk* |
|  6   |   writeBlock* received exception*   |       |  6   |   writeBlock* received exception*   |
|  7   | PktResponder* for blk* Interrupted. |       |  7   | PktResponder* 

/Users/ajayshewale/SoK-Security/deepaid/timeseries_onehot.py:61: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  loss_accuracy = Bound(self.bound_thres-Logit(out)[label[0]])
/Users/ajayshewale/SoK-Security/deepaid/timeseries_onehot.py:74: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if torch.max(Logit(out)).cpu().data > self.pos_thres:
/Users/ajayshewale/SoK-Security/deepaid/timeseries_onehot.py:77: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  IDX2.append(torch.argmax(Logit(out)).cpu().data.numpy().tolist())


## Run XAI methods from Captum

In [8]:
from captum.attr import IntegratedGradients, InputXGradient, DeepLift, GradientShap, GradientAttribution, Lime, Occlusion, KernelShap, Saliency

In [9]:
# Will store results for all the XAI methods which we ran
df = pd.DataFrame()

In [11]:
# Helper Function to get the weight value
def get_weights(attr, anomaly_timeseries):
    attr_weights = []
    for index, val in enumerate(attr):
    #     print(anomaly_timeseries[index])
        anom_val = anomaly_timeseries[index]
        # get the anomaly_timeseries value
        temp_wieght = val[anom_val].item()
        attr_weights.append(temp_wieght)
    
    return attr_weights

## Gradient

In [132]:
saliency = Saliency(model)

saliency_attributions = saliency.attribute(seq.unsqueeze(0), target=label)
saliency_attr = saliency_attributions[0].to(torch.float32)

/Users/ajayshewale/miniforge3/envs/explainable_env/lib/python3.10/site-packages/captum/_utils/gradient.py:57: UserWarning: Input Tensor 0 did not already require gradients, required_grads has been set automatically.
  warnings.warn(


In [133]:
saliency_attr_weights = get_weights(saliency_attr, anomaly_timeseries)

In [134]:
df["Gradient"] = saliency_attr_weights

In [12]:
df.head()

""


## InputXGradient

In [71]:
inputGrad = InputXGradient(model)

inputGrad_attributions = inputGrad.attribute(seq.unsqueeze(0), target=label)
inputGrad_attr = inputGrad_attributions[0].to(torch.float32)

/Users/ajayshewale/miniforge3/envs/explainable_env/lib/python3.10/site-packages/captum/_utils/gradient.py:57: UserWarning: Input Tensor 0 did not already require gradients, required_grads has been set automatically.
  warnings.warn(


In [72]:
inputGrad_attr_weights = get_weights(inputGrad_attr, anomaly_timeseries)

In [73]:
df["inputGrad"] = inputGrad_attr_weights

In [84]:
df

,inputGrad,IntegratedGradients,deepLift
0,1.108235,-0.670030,1.108236
1,0.436778,0.488253,0.436778
2,-0.196666,0.084237,-0.196666
3,-0.060618,-0.003849,-0.060618
4,0.122957,0.052001,0.122957
5,-0.018185,-0.097406,-0.018185
6,0.422438,0.166666,0.422438
7,-0.042346,-0.043780,-0.042346
8,0.029296,0.007565,0.029296
9,0.048154,0.040908,0.048154


## IntegratedGradients

In [75]:
ig = IntegratedGradients(model)

ig_attributions = ig.attribute(seq.unsqueeze(0), target=label)
ig_attr = ig_attributions[0].to(torch.float32)

In [76]:
ig_attr_weights = get_weights(ig_attr, anomaly_timeseries)

In [77]:
df["IntegratedGradients"] = ig_attr_weights

## DeepLift

In [79]:
deepLift = DeepLift(model)

deepLift_attributions = deepLift.attribute(seq.unsqueeze(0), target=label)
deepLift_attr = deepLift_attributions[0].to(torch.float32)

/Users/ajayshewale/miniforge3/envs/explainable_env/lib/python3.10/site-packages/captum/_utils/gradient.py:57: UserWarning: Input Tensor 0 did not already require gradients, required_grads has been set automatically.
  warnings.warn(
/Users/ajayshewale/miniforge3/envs/explainable_env/lib/python3.10/site-packages/captum/attr/_core/deep_lift.py:304: UserWarning: Setting forward, backward hooks and attributes on non-linear
               activations. The hooks and attributes will be removed
            after the attribution is finished
  warnings.warn(


In [80]:
deepLift_attr_attr_weights = get_weights(deepLift_attr, anomaly_timeseries)

In [81]:
df["deepLift"] = deepLift_attr_attr_weights

In [ ]:
## Models to run
# gradient, inputXgrad, ig, deeplift, ,oclusion, lime, kernelshap, gradientshap

## Lime

In [92]:
lime = Lime(model)

lime_attributions = lime.attribute(seq.unsqueeze(0), target=label)
lime_attr = lime_attributions[0].to(torch.float32)

In [93]:
lime_attr_weights = get_weights(lime_attr, anomaly_timeseries)

In [94]:
df["lime"] = lime_attr_weights

## KernelShap

In [86]:
kernalshap = KernelShap(model)

kernalshap_attributions = kernalshap.attribute(seq.unsqueeze(0), target=label)
kernalshap_attr = kernalshap_attributions[0].to(torch.float32)

In [87]:
kernalshap_attr_weights = get_weights(kernalshap_attr, anomaly_timeseries)

In [88]:
df["kernalshap"] = kernalshap_attr_weights

## gradientshap

In [102]:
# define the baselines for GradientShap
baselines = torch.zeros_like(seq.unsqueeze(0))

In [103]:
gradientshap = GradientShap(model)

In [104]:
gradientshap_attributions = gradientshap.attribute(seq.unsqueeze(0), baselines, target=label)
gradientshap_attr = gradientshap_attributions[0].to(torch.float32)

In [105]:
gradientshap_attr_weights = get_weights(gradientshap_attr, anomaly_timeseries)

In [106]:
df["gradientshap"] = gradientshap_attr_weights

## Occulsion

In [116]:
occlusion = Occlusion(model)
window_size = 5
occlusion_attributions = occlusion.attribute(seq.unsqueeze(0),sliding_window_shapes=(window_size,1), target=label)
occlusion_attr = occlusion_attributions[0].to(torch.float32)

In [117]:
occlusion_attr_weights = get_weights(occlusion_attr, anomaly_timeseries)

In [118]:
df["occlusion"] = occlusion_attr_weights

### add event details column in our result df 

In [123]:
# Add Event name
event_name_col = ["Receiving blk* src&dest:* ", "PktResponder* for blk* terminating", "PktResponder* Exception", "Exception in receiveBlock for blk*", "writeBlock* received exception*", "PktResponder* for blk* Interrupted", "PktResponder* for blk* terminating", "Exception in receiveBlock for blk*", "writeBlock* received exception*", "PktResponder* for blk* terminating"]

In [124]:
df["Event Details"] = event_name_col

In [143]:
df = df.reindex(columns=['Event Details','Gradient', 'inputGrad', 'IntegratedGradients', 'deepLift', 'lime', 'kernalshap', 'occlusion','gradientshap'])

In [146]:
df.columns

Index(['Event Details', 'Gradient', 'inputGrad', 'IntegratedGradients',
       'deepLift', 'lime', 'kernalshap', 'occlusion', 'gradientshap'],
      dtype='object')

In [15]:
df.head()

""


In [156]:
# Removing gradientshap as we dont want it in our final result
df = df.drop(columns=['gradientshap'])

In [157]:
df.head()

,Event Details,Gradient,inputGrad,IntegratedGradients,deepLift,lime,kernalshap,occlusion
0,Receiving blk* src&dest:*,1.108235,1.108235,-0.670030,1.108236,-0.524080,-0.007913,-0.531507
1,PktResponder* for blk* terminating,0.436778,0.436778,0.488253,0.436778,0.149037,0.038478,0.546340
2,PktResponder* Exception,0.196666,-0.196666,0.084237,-0.196666,0.000000,0.010203,-0.118946
3,Exception in receiveBlock for blk*,0.060618,-0.060618,-0.003849,-0.060618,0.000000,0.007081,-0.078701
4,writeBlock* received exception*,0.122957,0.122957,0.052001,0.122957,-0.008380,0.003667,0.118297


## Save the results

In [158]:
# Save the final result csv
df.to_csv("captum_results.csv", index=False)

### END

In [46]:
## Models to run
# gradient, inputXgrad, ig, deeplift, ,oclusion, lime, kernelshap, gradientshap

In [45]:
# """Step 3: Create a DeepAID Interpreter"""

# from timeseries_onehot import UniTimeseriesAID
# feature_desc = np.load('deepaid/log_key_meanning.npy') # feature_description
# my_interpreter = UniTimeseriesAID(model, feature_desc=feature_desc, class_num=28)

# """Step 4: Interpret your anomaly and show the result"""
# interpretation = my_interpreter(attr, label)
# my_interpreter.show_table(anomaly_timeseries, interpretation)

In [18]:
# input = torch.tensor(abnormal_data, dtype=torch.float32, requires_grad=True)

# attribution_scores, _ = ig.attribute(input, target=9, return_convergence_delta=True)

# """ Deeplog tools with Integrated Gradients"""
# def deeplogtools_seqformat_IG(model, abnormal_data, num_candidates, index=0):
#     import keras.utils.np_utils as np_utils
#     from captum.attr import IntegratedGradients

#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#     X = abnormal_data.copy()
#     y, X = X[:,-1], np_utils.to_categorical(X[:,:-1])
#     Output = model(torch.from_numpy(X).type(torch.float).to(device))

#     # Calculate Integrated Gradients
#     ig = IntegratedGradients(model)

#     TP_idx = []
#     for i in range(len(Output)):
#         output = Output[i]
#         label = y[i]
#         predicted = torch.argsort(output)[-num_candidates:]
#         if label not in predicted:
#             TP_idx.append(i)
#     seq_feat = np_utils.to_categorical(abnormal_data[TP_idx])
#     feat = seq_feat[index]
#     seq = torch.from_numpy(feat[:-1,:]).to(device)
#     label = torch.tensor(np.argmax(feat[-1])).unsqueeze(0).to(device)

#     # Calculate Integrated Gradients for the input sequence
#     seq_ig = ig.attribute(seq, target=label)
    
#     return seq_ig, label, abnormal_data[TP_idx][index]

# deeplogtools_seqformat_IG(model, abnormal_data, 10)